In [1]:
import json
eval_data = json.load(open("C:\\Users\\PC\\CODE\\WDM-AI-TEMIS\\data-finetune\\final_data\\docling_json.json", "r", encoding="utf-8"))

In [2]:
idx = 6

ground_truth = eval_data[idx]["markdown_content"]
predict = eval_data[idx]["docling_md"]

In [3]:
ground_truth

'|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|\n|--|--|--|--|--|--|--|--|--|--|--|--|--|--|\n|Month| Jan| Feb| Mar| Apr| May| Jun| Jul| Aug| Sep| Oct| Nov| Dec| Year|\n|Mean dailymaximum °C (°F)|26(79)|27(81)|25(77)|23(73)|20(68)|17(63)|17(63)|20(68)|23(73)|27(81)|27(81)|28(82)|23(74)|\n|Mean dailyminimum °C (°F)|15(59)|15(59)|13(55)|6(43)|6(43)|3(37)|1(34)|0(32)|5(41)|12(54)|14(57)|16(61)|9(48)|\n|Average rainfallmm (inches)|76(3.0)|98(3.9)|55(2.2)|28(1.1)|50(2.0)|34(1.3)|0(0)|0(0)|0(0)|22(0.9)|103(4.1)|68

In [4]:
predict

'| Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea District   | [hide]        |\n|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|-----------------------------------|---------------|\n| Month                  

In [5]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("Lỗi: OPENAI_API_KEY không được tìm thấy.")
else:
    llm = ChatOpenAI(
        api_key=OPENAI_API_KEY,
        model="gpt-4o-mini",
        temperature=0,
        model_kwargs={
            "response_format": {"type": "json_object"}
        }
    )
    print("DEBUG: LLM initialized with JSON mode.")
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from langchain_core.output_parsers import JsonOutputParser

def evaluate_table_extraction(ground_truth: str, predict: str, llm_model = None, debug: bool = False) -> any: # Thay đổi kiểu trả về nếu cần
   parser = JsonOutputParser()

   prompt_template_str = """
   You are provided with two markdown tables: a ground truth table and a predicted table.

Ground Truth Table:

{ground_truth}

Predicted Table:

{predict}

Evaluate the similarity between the two tables on a scale of 0 to 1, where 0 indicates no similarity and 1 indicates perfect match. For table fragments (tables without headers), prioritize the content and structure of the tables over the header values.

Provide a similarity score and a brief explanation of your reasoning.
You must respond with a JSON object containing only a score field with a float value between 0 and 1.
Remember: Your entire response must be a valid JSON object with only a "score" field containing a number between 0 and 1.
   """
   prompt = PromptTemplate.from_template(template=prompt_template_str)

   try:
      chain = prompt | llm_model | parser
      if debug:
         print("DEBUG: Chain created successfully.")

      invocation_payload = {
         "ground_truth": ground_truth,
         "predict": predict
      }
      if debug:
         print(f"DEBUG: Invoking chain with payload (first 100 chars of each): \nGround truth: {ground_truth[:100]}...\nPredict: {predict[:100]}...")

      # Để kiểm tra riêng lẻ LLM (bỏ qua parser tạm thời):
      # formatted_prompt = prompt.invoke(invocation_payload)
      # print(f"DEBUG: Formatted prompt sent to LLM:\n{formatted_prompt.to_string()}") # Hoặc .text tùy phiên bản
      # llm_output = llm_model.invoke(formatted_prompt)
      # print(f"DEBUG: Raw output from LLM:\n{llm_output}")
      # res = parser.parse(llm_output) # Parse thủ công nếu muốn kiểm tra parser

      res = chain.invoke(invocation_payload)
      if debug:
         print(f"DEBUG: Chain invocation successful. Result (res): {res}")
         print(f"DEBUG: Type of res: {type(res)}")
      return res

   except Exception as e:
      if debug:   
         print(f"ERROR in evaluate_table_extraction: {e}")
         import traceback
         traceback.print_exc() # In ra chi tiết lỗi và dòng gây lỗi
      return "" # Hoặc None, hoặc một giá trị báo lỗi cụ thể

DEBUG: LLM initialized with JSON mode.


In [6]:
a = evaluate_table_extraction(ground_truth, predict, llm, debug=True)
print(a)

DEBUG: Chain created successfully.
DEBUG: Invoking chain with payload (first 100 chars of each): 
Ground truth: |Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea ...
Predict: | Climate data for Berea District   | Climate data for Berea District   | Climate data for Berea Dis...
DEBUG: Chain invocation successful. Result (res): {'score': 0.85}
DEBUG: Type of res: <class 'dict'>
{'score': 0.85}


In [7]:
from tqdm import tqdm
total_score = 0


for sample in tqdm(eval_data):
    ground_truth = sample['markdown_content']
    predict = sample['docling_md']
    score = evaluate_table_extraction(ground_truth, predict, llm)
    total_score += score['score']
    # print(score)
print(total_score / len(eval_data))


  0%|          | 0/163 [00:00<?, ?it/s]

100%|██████████| 163/163 [02:46<00:00,  1.02s/it]

0.7034969325153373


In [ ]:
from tqdm import tqdm
from gemini import *
from gemini import generate
import time

total_score = 0
max_retries = 3  # Maximum number of retries for handling connection errors

for sample in tqdm(eval_data):
    ground_truth = sample['markdown_content']
    predict = sample['docling_md']
    retries = 0
    while retries < max_retries:
        try:
            score = generate(ground_truth, predict)
            total_score += score
            break  # Exit the retry loop if successful
        except ConnectionResetError as e:
            print(f"Connection error: {e}. Retrying {retries + 1}/{max_retries}...")
            retries += 1
            time.sleep(2)  # Wait for 2 seconds before retrying
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break  # Exit the retry loop on unexpected errors

if len(eval_data) > 0:
    print(total_score / len(eval_data))
else:
    print("No data to evaluate.")


 10%|▉         | 16/163 [00:47<07:20,  3.00s/it]